In [1]:
"""

190 page 기준 8분 정도
휴일 때 대략 100~200 page
평일 때 대략 400~500 page 


한 페이지에 15 기사

"""

'\n\n190 page 기준 8분 정도\n휴일 때 대략 100~200 page\n평일 때 대략 400~500 page \n\n\n한 페이지에 15 기사\n\n'

In [2]:
from datetime import date
import pandas as pd
from tqdm import notebook
import requests
from bs4 import BeautifulSoup
import re



In [3]:
def create_soup(url):

    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.content, 'html.parser', from_encoding='cp949')
    return soup

In [16]:
def get_articles(DATE, category, save_today = False):
    from datetime import date
    import pandas as pd
    from tqdm import notebook
    import requests
    from bs4 import BeautifulSoup
    
    result = pd.DataFrame(columns=['arcid', 'press', 'title', 'content', 'journalist', 'date', 'link', 'category'])
    
    press_using_list = ["경향신문", "국민일보", "뉴스1", "뉴시스", "동아일보", "문화일보", "서울신문", "세계일보", "연합뉴스",  "조선일보", "중앙일보", "한겨레", "한국일보"]
    no_get_list = ["@","무단전재", "재배포금지","저작권자 ⓒ 서울신문사","무단복제 및 전재","무단 전재 및 재배포","제보는 카톡", "☞", "무단 전재-재배포", "▶연합뉴스 앱 지금 바로 다운받기~"]
    replace_list = ["\xa0", "\u2008"]
    
    
    articleid_list = []
    press_list = []
    title_list = []
    content_list = []
    journalist_list = []
    link_list = []

    
    
    # date
    year = str(DATE.year)
    month = str(DATE.month).zfill(2)
    day = str(DATE.day).zfill(2)
    date_today = year+month+day
    
    
    # 마지막 페이지 번호 추출
    url = f"https://news.daum.net/breakingnews/{category}?page=999&regDate={date_today}"
    soup = create_soup(url)
    last_page = soup.find("em",attrs ={"class": "num_page"})
    last_page_num = int(re.sub(r'[^0-9]', '', last_page.text))
    
   
    for i in notebook.tqdm(range(1,last_page_num+1)):
        

        url = f"https://news.daum.net/breakingnews/{category}?page={i}&regDate={date_today}"
        soup = create_soup(url)
        
        
        # 뉴스별 정보(링크, 언론사 포함된) 리스트
        news_list_part = soup.find('ul', attrs={'class':'list_news2 list_allnews'}).find_all("li")


        for news_info in news_list_part:
            
            ### 기사 예외처리 ################################################################
            
            #  press 예외 처리
            press = news_info.find("span", attrs = {"class":"info_news"}).text.split()[0]
            
            
            if press not in press_using_list:
                continue
            
            link = news_info.select_one("strong>a")["href"]
            article_soup = create_soup(link).find("article")
            
            
            # journalist 예외 처리 
            journalist = article_soup.find("span", attrs={'class':"txt_info"}).text
            if journalist.split()[0] == "입력":
                continue
                
            ################################################################ 기사 예외처리  ###
            
            ### press
            ### link
            ### journalist
            press_list.append(press) 
            link_list.append(link)    
            journalist_list.append(journalist)
            
            
            ### articleid
            articleid_list.append(link.split("/")[-1])

                
            ### title
            title = article_soup.find("h3", attrs={'class':"tit_view"})
            title_list.append(title.text)
            
            
            ### 본문 추출 ######################################################################
            content_soup = article_soup.find("div", attrs={'class':"article_view"})
    
            # 이미지 제거
            if content_soup.select_one("p.link_figure") != None:
                content_soup.select_one("p.link_figure").decompose()
                
            
            content_part_list = content_soup.find_all(True, attrs={'dmcf-ptype':'general'})
            content = ""
            
            # 
            for content_part in content_part_list:
                skip_content_part = False
                
                tmp = content_part.text
                
                # 예외가 포함된 문장을 건너뛰기
                for no_get in no_get_list :
                    if no_get in tmp :
                        skip_content_part = True
                        break;
                        
                if skip_content_part :
                    continue
                    
                # 주로 \과 관련된 문자를 ""로 변경
                for replace_word in replace_list:
                    tmp = tmp.replace(replace_word,"")
                    
                    
                tmp = tmp.strip()

                        
                content += tmp
            ######################################################################### 본문 추출 ###
                
            content_list.append(content)
            


    result['arcid'] = articleid_list
    result['press'] = press_list
    result['title'] = title_list
    result['content'] = content_list
    result['journalist'] = journalist_list
    result['link'] = link_list
    
    
    result['date'] = DATE
    result['category'] = category
    
    if save_today:
        path = "data/"
        file_name = date_today + "_" + category
        df.to_csv(path+file_name)
    
    return result

In [8]:
# [카카오톡] YTN을 검색해 채널 추가 [전화] 02-398-8585 [메일] social@ytn.co.kr [온라인 제보] www.ytn.co.kr

In [9]:
from dateutil.rrule import rrule, DAILY

df = pd.DataFrame()
start_date = date(2022, 1, 1)
end_date = date(2022, 1, 15)

for date_ in notebook.tqdm(list(rrule(DAILY, dtstart=start_date, until=end_date))):
    df = pd.concat([df, get_articles(date,"society")])
df.reset_index(drop=True, inplace=True)


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


KeyboardInterrupt



In [17]:
DATE = date(2022, 1, 1)
df = get_articles(DATE,"society",True)

  0%|          | 0/190 [00:00<?, ?it/s]

In [40]:
# """
# HTTPSConnectionPool(host='v.daum.net', port=443): Max retries exceeded with url: /v/20220101121016727 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000020C9A0B23A0>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))
# """


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1607 entries, 0 to 1606
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   arcid       1607 non-null   object
 1   press       1607 non-null   object
 2   title       1607 non-null   object
 3   content     1607 non-null   object
 4   journalist  1607 non-null   object
 5   date        1607 non-null   object
 6   link        1607 non-null   object
 7   category    1607 non-null   object
dtypes: object(8)
memory usage: 100.6+ KB


In [31]:
list(df["journalist"].value_counts().index)

['임세영 기자',
 '하경민',
 '신웅수 기자',
 '이영환',
 '박지혜 기자',
 '이지은',
 '윤일지 기자',
 '차용현',
 '최창호 기자',
 '전원 기자',
 '양지웅',
 '이종건',
 '김동민',
 '유형재',
 '홍수영 기자',
 '김주성',
 '조성우',
 '김용태',
 '황기선 기자',
 '박주영',
 '변재훈',
 '윤왕근 기자',
 '노경민 기자',
 '김영운 기자',
 '김얼',
 '김현태',
 '조남수',
 '최인영',
 '김동영',
 '유경석 기자',
 '강신욱',
 '옥성구',
 '홍정명',
 '이연희',
 '남승렬 기자',
 '배병수',
 '진연수',
 '박제철 기자',
 '배훈식',
 '구미현',
 '송창헌',
 '김혜인',
 '고석중',
 '손형주',
 '김재현',
 '강대한 기자',
 '김정호 기자',
 '김종효',
 '이무열',
 '정진욱 기자',
 '유승관 기자',
 '변지철',
 '최대호 기자',
 '권창회',
 '조명휘',
 '이진욱',
 '정회성',
 '성동훈 기자',
 '김상연',
 '임명수',
 '김진호',
 '이종익',
 '박홍식',
 '이시우 기자',
 '김선경',
 '신관호 기자',
 '이승연',
 '김유민',
 '김진아',
 '이동민',
 '양다훈',
 '전준우 기자',
 '홍기원',
 '이바름',
 '한종찬',
 '음상준 기자',
 '안지율',
 '차근호',
 '강릉 | 이준헌 기자',
 '강명수',
 '고석현',
 '이덕화',
 '윤난슬',
 '한훈',
 '강민선',
 '유희곤 기자',
 '이상학',
 '안정원',
 '박태근 동아닷컴 기자',
 '김동수 기자',
 '최서영 기자',
 '한송학 기자',
 '이영주',
 '김도엽 기자',
 '신재현',
 '한영혜',
 '임효진',
 '박종대',
 '강교현 기자',
 '권형진 기자',
 '김민기 기자',
 '이루비',
 '임기창',
 '박슬용 기자',
 '유순상',
 '김대광 기자',
 '조준영 기자',
 '이보람',
 '박종국',
 '김형우',
 '

In [32]:
df["press"].value_counts()

뉴스1     551
뉴시스     450
연합뉴스    377
조선일보     48
세계일보     36
중앙일보     28
서울신문     27
한국일보     25
동아일보     21
경향신문     18
국민일보     13
한겨레      10
문화일보      3
Name: press, dtype: int64

In [33]:
df.head()

,arcid,press,title,content,journalist,date,link,category
0,20220101235811959,세계일보,"""우리 촉법소년들이라고!"" 경찰관 폭행한 10대들, 되레 인권위 진정 제기",술집에서 술을 마시던10대 미성년자들이 단속을 나온 경찰관에게 폭력을 휘두른 것도 ...,현화영,2022-01-01,https://v.daum.net/v/20220101235811959,society
1,20220101235510951,중앙일보,"문자 한통에..4시간 벌벌 떨며 검사받은 초.중학생들, 무슨일",새해 첫날인 1일 광주지역 학교 2곳의 전수검사가 이뤄지면서 학생과 그 가족이 몰린...,한영혜,2022-01-01,https://v.daum.net/v/20220101235510951,society
2,20220101232414807,뉴스1,시민 몰린 광주시청 선별진료소,(광주=뉴스1) 전원 기자 = 1일 오후 광주시청 앞 선별진료소에 시민들의 발길이 ...,전원 기자,2022-01-01,https://v.daum.net/v/20220101232414807,society
3,20220101232328801,뉴스1,검사받으러는 시민 몰린 광주시청 선별진료소,(광주=뉴스1) 전원 기자 = 1일 오후 광주시청 앞 선별진료소에 시민들의 발길이 ...,전원 기자,2022-01-01,https://v.daum.net/v/20220101232328801,society
4,20220101231602763,서울신문,"경찰 목 조른 10대들 ""인권침해다..우린 촉법소년""(영상)",10대 미성년자들이 술집에서 술을 마시다 단속 경찰에게 폭력을 휘두른 것도 모자라 ...,김유민,2022-01-01,https://v.daum.net/v/20220101231602763,society


In [35]:
for i in range(100):
    print(f"""=================== {i}  {df.iloc[i]["journalist"]}  {df.iloc[i]["link"]}  ================================""")
    print(df.iloc[i]["content"])

=================== 0  현화영  https://v.daum.net/v/20220101235811959  ================================
술집에서 술을 마시던10대 미성년자들이 단속을 나온 경찰관에게 폭력을 휘두른 것도 모자라, 되레 ‘인권 침해를 당했다’며 국가인권위원회에 진정을 제기한 사실이 뒤늦게 알려졌다. 이들은 10대 후반이라 촉법소년에 해당하지 않는다고 경찰 측은 설명했다.지난 1일 서울 서부경찰서에 따르면,지난해11월29일 오전12시40분쯤 10대 청소년 8명이은평구의 한 주점에서 술을 마시다 경찰관의 목을 조르고 주먹을 휘둘러 4명이 공무집행방해 등 혐의로 입건되는 사건이 발생했다.그런데 당시 입건된 4명 중 3명은 자신들이‘촉법소년’이라며 한 달이 지나도록 경찰 출석 요구에 응하지 않고 있는 것으로 전해졌다.이 뿐만 아니라 “사건 당시 현장에서 경찰이 과잉진압을 했다”라며국가인권위원회에 진정까지 냈다.하지만 촉법소년은 ‘범죄 행위를 한10세 이상14세 미만 소년’으로,10대 후반인 이들에게는 적용되지 않으며 형사처벌이 가능한 것으로 전해졌다.촉법소년은 이른바 ‘형사 미성년자’를 말한다. 형법은 “14세가 되지 않은 자의 행위는 벌하지 않는다”고 규정하고 있어 촉법소년에게는 형사처벌 대상이 될 수 없다.채널A는 사건 당시 모습이 찍힌 폐쇄회로(CC)TV영상을 공개했다.이 영상에는 경찰과 미성년자가 몸싸움을 하며 술집 밖으로 나오는 장면이 담겼다. 10대들은 경찰관들의 목을 조르고, 얼굴에 주먹을 휘둘렀다.경찰관의 다리를 잡아 넘어뜨리기도 했다.결국 경찰이 전기충격기 등을 이용해 폭력을 행사한10대 남성 3명을 긴급 체포했다.이들은 경찰 지구대에서도 자신들은 촉법소년이라고 주장한 것으로 전해졌다.당시 목격자는“(현장에 있던 여성들이) 남자친구들 못 데려가게 몸으로 막으며‘인권침해다, 신고한다’고 막 전화기를 꺼냈다”면서 “(10대들이) 욕설 하니까 경찰이 ‘욕하지 마세요’라고 했다”고 전했다.경찰은 이들 

In [ ]:
 ["경향신문", "국민일보", "뉴스1", "뉴시스", "동아일보", "문화일보", "서울신문", "세계일보", "연합뉴스",  "조선일보", "중앙일보", "한겨레", "한국일보"]

In [ ]:
df.iloc[552]["link"]
# [447:451] [552]

In [ ]:
df[df["journalist"] == "한겨레"]["link"]

In [ ]:
from dateutil.rrule import rrule, DAILY

df = pd.DataFrame()

for date_ in notebook.tqdm(list(rrule(DAILY, dtstart=start_date, until=end_date))):
    df = pd.concat([df, get_articles(date,"society",press_using_list)])
df.reset_index(drop=True, inplace=True)


In [36]:
df_test = pd.read_csv("data/20220101_society")

In [37]:
df_test

,Unnamed: 0,arcid,press,title,content,journalist,date,link,category
0,0,20220115234101239,서울신문,붕괴사고 닷새째 수색 종료..낙하물 추락에 한때 수색 중단도,위험 요인에 타워크레인 해체 ‘작업중지권’ 발동…장기화 국면광주 신축 아파트 구조물...,이범수,2022-01-15,https://v.daum.net/v/20220115234101239,society
1,1,20220115232710154,뉴스1,전주 상가 건물서 불..50대 2명 숨진 채 발견,(전북=뉴스1) 강교현 기자 = 15일 오후 5시43분께 전북 전주시 중화산동의 한...,강교현 기자,2022-01-15,https://v.daum.net/v/20220115232710154,society
2,2,20220115232451147,조선일보,"""박원순, 성범죄 저지르고 극단선택"" 발언 만화가 윤서인 불송치",고(故) 박원순 전 서울시장에 대해 “성범죄 저지르고 자살한 사람”이라고 언급해 유...,김명일 기자,2022-01-15,https://v.daum.net/v/20220115232451147,society
3,3,20220115231149073,뉴시스,[부고] 한준열(농협 경남본부 경영기획단 과장보)씨 부친상,"[창원=뉴시스] ▲한상욱씨 별세, 이보애씨 남편상, 한준열(NH농협중앙회 경남지역본...",홍정명,2022-01-15,https://v.daum.net/v/20220115231149073,society
4,4,20220115230927062,조선일보,"""수색 더뎌 한없이 죄송"" 이용섭 광주시장, 붕괴사고 사과문",이용섭 광주시장이 신축아파트 외벽 붕괴사고와 관련해 “실종자와 가족들 그리고 시민들...,문지연 기자,2022-01-15,https://v.daum.net/v/20220115230927062,society
5,5,20220115230500049,뉴스1,홍성 이호저수지서 낚시하던 70대 남성 숨져,(홍성=뉴스1) 김낙희 기자 = 충남 홍성군 서부면 이호저수지에서 낚시를 하던 70...,김낙희 기자,2022-01-15,https://v.daum.net/v/20220115230500049,society
6,6,20220115224601950,서울신문,오후 9시까지 전국서 3552명 확진..일주일 전보다 450명↑,경기 1210명·서울 885명·인천 260명 등 수도권 2355명오미크론 변이가 확...,이범수,2022-01-15,https://v.daum.net/v/20220115224601950,society
7,7,20220115223813914,연합뉴스,전주 상가 건물서 불..50대 2명 숨진 채 발견,(전주=연합뉴스) 최영수 기자 = 15일 오후 5시 43분께 전북 전주시 완산구 중...,최영수,2022-01-15,https://v.daum.net/v/20220115223813914,society
8,8,20220115223006860,문화일보,코로나 먹는치료제 효과는?.. 증상호전 환자가 다수,"첫 투약자는 70대 대전 남성, 14일 전국에서 9명 처방‘코로나 19 먹는 치료제...",인지현 기자,2022-01-15,https://v.daum.net/v/20220115223006860,society
9,9,20220115222623829,연합뉴스,붕괴사고 닷새째 수색 종료..낙하물 추락에 한때 수색 중단도(종합),(광주=연합뉴스) 정회성 기자 = 광주 신축 아파트 구조물 붕괴 사고 현장에서 실종...,정회성,2022-01-15,https://v.daum.net/v/20220115222623829,society
